# Lecture 9: Geometric Camera Models

CMU 16-385 Computer Vision, Fall 2020 (http://16385.courses.cs.cmu.edu/)

In [75]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

range = 6

# Define a 3D box (8 vertices, 12 edges)
vertices = np.array([[-1, -1, -1,  1], \
                     [-1, -1,  1,  1],
                     [-1,  1, -1,  1],
                     [-1,  1,  1,  1],
                     [ 1, -1, -1,  1],
                     [ 1, -1,  1,  1],
                     [ 1,  1, -1,  1],
                     [ 1,  1,  1,  1]])
vertices = np.transpose(vertices)
edges = np.array([[0, 1], [2, 3], [4, 5], [6, 7], \
                  [0, 2], [1, 3], [4, 6], [5, 7], \
                  [0, 4], [1, 5], [2, 6], [3, 7]])

def h(rx = 0.0, ry = 0.0, rz = 0.0, tz = 4.0, f = 100.0):
  
    # DEFINE EXTRINSICS

    # Rotation about Z axis
    rotz = np.array([[np.cos(rz), -np.sin(rz),  0,          0], \
                     [np.sin(rz),  np.cos(rz),  0,          0], \
                     [0,           0,           1,          0], \
                     [0,           0,           0,          1]])
    
    # Rotation about Y axis
    roty = np.array([[np.cos(ry),  0,          -np.sin(ry), 0], \
                     [0,           1,           0,          0], \
                     [np.sin(ry),  0,           np.cos(ry), 0], \
                     [0,           0,           0,          1]])
    
    # Rotation about X axis
    rotx = np.array([[1,           0,           0,          0], \
                     [0,           np.cos(rx), -np.sin(rx), 0], \
                     [0,           np.sin(rx),  np.cos(rx), 0], \
                     [0,           0,           0,          1]])

    # Rotation along Z axis
    traz = np.array([[1,           0,           0,          0], \
                     [0,           1,           0,          0], \
                     [0,           0,           1,          tz], \
                     [0,           0,           0,          1]])
    
    # Perspective Projection
    proj = np.array([[1,           0,           0,          0], \
                     [0,           1,           0,          0], \
                     [0,           0,           1,          0]])
    
    extrinsics = rotz
    extrinsics = np.matmul(roty, extrinsics)
    extrinsics = np.matmul(rotx, extrinsics)
    extrinsics = np.matmul(traz, extrinsics)
    extrinsics = np.matmul(proj, extrinsics)

    # DEFINE INTRINSICS

    intrinsics = np.array([[f, 0, 50], \
                           [0, f, 50], \
                           [0, 0, 1]])

    # Transform points
    pts3D = np.matmul(extrinsics, vertices)
    pts2D = np.matmul(intrinsics, pts3D)

    # Homogeneous coordinate -> Heterogeneous coordinate
    pts2D[0,:] = pts2D[0,:] / pts2D[2,:]
    pts2D[1,:] = pts2D[1,:] / pts2D[2,:]

    # Compute camera's view frustum
    corners = [[0, 0, 0], [0, 0, 1], [0, 100, 1], [100, 100, 1], [100, 0, 1]]
    corners = np.transpose(corners)
    corners = np.matmul(np.linalg.inv(intrinsics),corners)

    for k in np.arange(4)+1:
      corners[:,k] = corners[:,k]*range/corners[2,k]

    # Plot results in 2D and 3D
    fig = plt.figure()
    ax_2D = fig.add_subplot(122)
    ax_3D = fig.add_subplot(121, projection='3d')

    # Plot cube
    for edge in edges:
      ax_2D.plot(pts2D[0,edge], pts2D[1,edge])
      ax_3D.plot(pts3D[0,edge], pts3D[2,edge], pts3D[1,edge])

    # Plot camera's view frustum
    ax_3D.plot(corners[0,[1,2,3,4,1]], corners[2,[1,2,3,4,1]], corners[1,[1,2,3,4,1]], 'k')
    for k in np.arange(4)+1:
      ax_3D.plot(corners[0,[0,k]], corners[2,[0,k]], corners[1,[0,k]], 'k')

    ax_2D.axis([0, 100, 0, 100])
    ax_3D.axes.set_xlim3d(left=-range/2, right=range/2) 
    ax_3D.axes.set_ylim3d(bottom=0, top=range) 
    ax_3D.axes.set_zlim3d(bottom=-range/2, top=range/2) 

interact(h, rx=(-np.pi, np.pi), ry=(-np.pi, np.pi), rz=(-np.pi, np.pi), tz=(3.0,6.0), f=(100,150));

interactive(children=(FloatSlider(value=0.0, description='rx', max=3.141592653589793, min=-3.141592653589793),…